# Verificación de la versión de python empleada

In [ ]:
import sys
print(f"Versión de Python: {sys.version}")


# Importar las librerías adicionales

In [ ]:
import sys
import os

# Añadir la carpeta correcta a sys.path
sys.path.insert(0, '/tmp/python/python/')

# Verificar que la ruta se ha agregado correctamente
print("Rutas actuales en sys.path:")
print(sys.path)

# Verificar el contenido de la carpeta python para confirmar que están las bibliotecas
print("Contenido de /tmp/python/python/:")
print(os.listdir('/tmp/python/python/'))


# Testear las librerías

In [ ]:
import requests
import lxml

print(f"requests versión: {requests.__version__}")
print(f"lxml versión: {lxml.__version__}")


# Selecionar los rangos de fechas con los que trabajar

In [ ]:
from datetime import datetime

# Definir las fechas de inicio y fin para la descarga histórica
fecha_inicio = "2024-09-01"  # Fecha de inicio en formato YYYY-MM-DD
fecha_fin = "2024-09-7"     # Fecha de fin en formato YYYY-MM-DD

# Convertir las fechas en objetos datetime
fecha_inicio_dt = datetime.strptime(fecha_inicio, '%Y-%m-%d')
fecha_fin_dt = datetime.strptime(fecha_fin, '%Y-%m-%d')

print(f"Descargando sumarios entre {fecha_inicio} y {fecha_fin}")


# Funciones de descarga de sumarios, contenidos trasformación y limpieza de los datos.

In [11]:
import requests
import boto3

# Inicializar cliente de S3
s3 = boto3.client('s3')
bucket_name = 'buckettestdescargas'
sumario_prefix = 'Descargas_sumarios/'

# Función para descargar el sumario para una fecha específica
def descargar_sumario(fecha):
    # Convertir la fecha al formato adecuado para la API
    fecha_str = fecha.strftime('%Y%m%d')
    url = f"https://www.boe.es/diario_boe/xml.php?id=BOE-S-{fecha_str}"
    
    try:
        # Descargar el sumario en formato XML
        response = requests.get(url)
        response.raise_for_status()
        sumario_xml = response.content

        # Subir el archivo a S3
        nombre_archivo = f"sumario_{fecha.strftime('%Y-%m-%d')}.xml"
        s3_key = f"{sumario_prefix}{nombre_archivo}"
        s3.put_object(Bucket=bucket_name, Key=s3_key, Body=sumario_xml, ContentType='application/xml')

        print(f"Sumario del {fecha.strftime('%Y-%m-%d')} descargado y almacenado en S3.")
        return s3_key

    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el sumario para la fecha {fecha.strftime('%Y-%m-%d')}: {e}")
        return None


In [ ]:
from datetime import timedelta

# Función para procesar un rango de fechas y descargar los sumarios
def procesar_rango_fechas(fecha_inicio, fecha_fin):
    fecha_actual = fecha_inicio

    while fecha_actual <= fecha_fin:
        print(f"Procesando fecha {fecha_actual.strftime('%Y-%m-%d')}")
        descargar_sumario(fecha_actual)
        fecha_actual += timedelta(days=1)

# Ejecutar la descarga para el rango de fechas
procesar_rango_fechas(fecha_inicio_dt, fecha_fin_dt)


In [ ]:
# Función para verificar los sumarios descargados en S3
def verificar_descargas(fecha_inicio, fecha_fin):
    fecha_actual = fecha_inicio

    while fecha_actual <= fecha_fin:
        nombre_archivo = f"sumario_{fecha_actual.strftime('%Y-%m-%d')}.xml"
        s3_key = f"{sumario_prefix}{nombre_archivo}"

        try:
            s3.head_object(Bucket=bucket_name, Key=s3_key)
            print(f"Sumario del {fecha_actual.strftime('%Y-%m-%d')} está presente en S3.")
        except:
            print(f"Sumario del {fecha_actual.strftime('%Y-%m-%d')} NO encontrado en S3.")

        fecha_actual += timedelta(days=1)

# Verificar los sumarios descargados
verificar_descargas(fecha_inicio_dt, fecha_fin_dt)


In [ ]:
# Función para eliminar los sumarios descargados en S3
def eliminar_archivos(fecha_inicio, fecha_fin):
    fecha_actual = fecha_inicio

    while fecha_actual <= fecha_fin:
        nombre_archivo = f"sumario_{fecha_actual.strftime('%Y-%m-%d')}.xml"
        s3_key = f"{sumario_prefix}{nombre_archivo}"

        try:
            s3.delete_object(Bucket=bucket_name, Key=s3_key)
            print(f"Sumario del {fecha_actual.strftime('%Y-%m-%d')} eliminado de S3.")
        except:
            print(f"Error al eliminar el sumario del {fecha_actual.strftime('%Y-%m-%d')} en S3.")

        fecha_actual += timedelta(days=1)

# Eliminar los archivos entre el rango de fechas
# eliminar_archivos(fecha_inicio_dt, fecha_fin_dt) # Descomenta esta línea si quieres eliminar los archivos
